## Initial steps

In [1]:
import map_functions

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

gdf = gpd.read_file("data/sources/Hungary_2022_static_road_data.geojson")

In [2]:
gdf_filled = gdf.replace([np.nan], [None])

## Combining roads: based on kszam (name)

Functions included in `map_functions.py`

In [3]:
gdf_new = map_functions.combine_roads_total_simple(gdf)

In [4]:
gdf_new[~gdf_new['min_5_traffic'].isna()]

,kszam,min_traffic,min_5_traffic,max_traffic,max_5_traffic,avg_traffic,geometry,data_df,data_json
0,M43,20728,20728,37030,34512,27256.641509,"MULTILINESTRING ((731774.360 106214.871, 73205...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
1,M4,3718,3718,57443,39233,20643.589744,"MULTILINESTRING ((848187.895 212117.248, 84819...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
2,M35,10875,10875,19402,18800,15349.763636,"MULTILINESTRING ((835825.477 255049.021, 83584...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3,M15,21871,26100,28451,28451,26436.523810,"MULTILINESTRING ((510618.213 297665.117, 51055...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
4,M44,8115,8851,9583,9583,9211.729167,"MULTILINESTRING ((800011.635 153748.361, 80293...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
...,...,...,...,...,...,...,...,...,...
3341,M8,7144,7144,13185,13185,10596.000000,"MULTILINESTRING ((637084.268 175766.926, 63722...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3342,M80,4712,5128,5828,5494,5358.944444,"MULTILINESTRING ((464853.113 189984.635, 46479...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3343,M9,4849,4849,4849,4849,4849.000000,"MULTILINESTRING ((626831.533 115238.519, 62690...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."
3344,2,5823,8282,35266,35266,17420.816667,"MULTILINESTRING ((656911.982 269457.874, 65690...",id kszam pkod kszelv vszelv ...,"{""type"": ""FeatureCollection"", ""features"": [{""i..."


## Reordering roads based on chaining roads together

In [5]:
gdf_reordered = map_functions.reorder_full_gdf_groupindexed(gdf)

c:\GitHubRepo\origin-destination-matrix-from-traffic\map_functions.py:183: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  gdf_reordered = pd.concat([gdf_reordered, gdf_group], ignore_index=True)


We get less roads as a result, because we exclude pkod '2' roads. <br>

In [6]:
#Let's create a dict: key would be the road name (kszam), whereas value would be the ordered list of road segments, with their Series values

roads_ordered = map_functions.create_ordered_roads_json(gdf)

In [9]:
(len(roads_ordered['3'][0]), len(roads_ordered['3'][1])), (len(gdf_reordered[(gdf_reordered['kszam']=='3') & (gdf_reordered['group']==0)]), len(gdf_reordered[(gdf_reordered['kszam']=='3') & (gdf_reordered['group']==1)]))

((50, 130), (50, 130))

In [ ]:
import json
import geojson

with open('data/Hungary_roads_ordered.geojson', 'w', encoding='utf-8') as f:
    geojson.dump(roads_ordered, f, ensure_ascii=False, indent=1)


## Plot roads nearby a place

In [1]:
import folium
#folium.Map(location=(47.130884, 16.621046))